In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np

The following code is to add drive to file path to extract files in google drive

In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass

!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144865 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.23-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.23-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.23-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

create a directory named 'drive' and link it

In [3]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [4]:
!ls


adc.json  drive  sample_data  test.csv	train.csv


Input training data

In [12]:
train_df = pd.read_csv('drive/Colab Notebooks/train.csv')
np.random.shuffle(train_df.values)
print(train_df.head())

          x         y  color
0  2.513819  3.599937    0.0
1  1.544729  2.422221    0.0
2  3.976169  4.823713    0.0
3  1.574778  0.429665    1.0
4  0.413381  1.190833    0.0


Neural Network

*See the keras documentaion for detailed explanation*


1.   **Sequential** - To define the whole network
2.   **Dense** - To define the layers
3.   **Compile** - To compile with proper optimizer and loss calcuation



In [20]:
model = keras.Sequential([
                          keras.layers.Dense(4, input_shape=(2,), activation='relu'),
                          keras.layers.Dense(2, activation='sigmoid')
])

model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
x = np.column_stack((train_df.x.values, train_df.y.values))
model.fit(x, train_df.color.values, batch_size=4, epochs = 5)

test_df = pd.read_csv('drive/Colab Notebooks/test.csv')
test_x = np.column_stack((test_df.x.values, test_df.y.values))

model.evaluate(test_x,test_df.color.values )

Epoch 1/5
1000/1000 [==============================] - 1s 847us/step - loss: 0.6344 - accuracy: 0.6967
Epoch 2/5
1000/1000 [==============================] - 1s 821us/step - loss: 0.5182 - accuracy: 0.9647
Epoch 3/5
1000/1000 [==============================] - 1s 815us/step - loss: 0.3813 - accuracy: 0.9980
Epoch 4/5
1000/1000 [==============================] - 1s 834us/step - loss: 0.3316 - accuracy: 0.9998
Epoch 5/5
32/32 [==============================] - 0s 888us/step - loss: 0.3198 - accuracy: 1.0000


[0.31982311606407166, 1.0]